#  Table detection using a CNN

In [1]:
%matplotlib inline
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import random

## Generate data

In [2]:
import random
def generateData(numImages=1,pageSize=(100,100),circleSize=(5,10)):
    imagesDraw = []
    imagesCNN = np.zeros((numImages,pageSize[0]*pageSize[1]),dtype=np.float32)
    labelsDraw = []
    labelsCNN = []

    for i in range(numImages):
        page = np.full(pageSize, 255, dtype=np.uint8)
        
        randSize = circleSize[0] + int(random.random()*(circleSize[1]-circleSize[0]+1))
        
        centreLoc = (randSize+int(random.random()*(pageSize[1]-randSize*2)), randSize+int(random.random()*(pageSize[0]-randSize*2)))
        cv2.circle(page, centreLoc,randSize, (0, 0, 255))

        imagesDraw.append(page)
        imagesCNN[i] = page.reshape(pageSize[0]*pageSize[1])
        
        recLoc = centreLoc[0]-randSize, centreLoc[1]-randSize, centreLoc[0]+randSize, centreLoc[1]+randSize
        
        labelsDraw.append((centreLoc,randSize))
        labelsCNN.append(recLoc)
    imagesCNN /= 255.0
    return imagesDraw,imagesCNN,labelsDraw,labelsCNN

In [3]:
circleDiam = (10,20)
pageSize = (100, 100)

circleSize = (int(circleDiam[0]/2),int(circleDiam[1]/2))
imagesDraw,imagesCNN,labelsDraw,labelsCNN = generateData(numImages=1000,pageSize=pageSize,circleSize=circleSize) #Do not make this more that around 1000

imagesDrawTrain = imagesDraw[:800]
imagesCNNTrain = imagesCNN[:800]
labelsDrawTrain = labelsDraw[:800]
labelsCNNTrain = labelsCNN[:800]

imagesDrawTest = imagesDraw[800:]
imagesCNNTest = imagesCNN[800:]
labelsDrawTest = labelsDraw[800:]
labelsCNNTest = labelsCNN[800:]

## Display the data (Causes kernel restart)

In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
x = tf.placeholder(tf.float32, shape=[None, pageSize[0]*pageSize[1]])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

## Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [7]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## First convolutional layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [8]:
Stack1Num = 10# Was 32
W_conv1 = weight_variable([5, 5, 1, Stack1Num]) #(window_dim1,window_dim1,input channels,output channels)
b_conv1 = bias_variable([Stack1Num])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

The first dimension is specified as -1 since it will be inferred from the number of images in the batch.

In [9]:
x_image = tf.reshape(x, [-1,pageSize[0],pageSize[1],1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.

In [10]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer



In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [11]:
Stack2Num = 20#Was 64
W_conv2 = weight_variable([5, 5, Stack1Num, Stack2Num])
b_conv2 = bias_variable([Stack2Num])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [12]:
size = 2500*5#7*7*64 #Was 7*7*64
W_fc1 = weight_variable([size, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, size])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer

Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [14]:
W_fc2 = weight_variable([1024, 4])
b_fc2 = bias_variable([4])

y_conv=tf.matmul(h_fc1_drop, W_fc2) + b_fc2#tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Train and Evaluate the Model

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network.

The differences are that:

We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.

We will include the additional parameter keep_prob in feed_dict to control the dropout rate.

We will add logging to every 100th iteration in the training process.

Feel free to go ahead and run this code, but it does 20,000 training iterations and may take a while (possibly up to half an hour), depending on your processor.



In [ ]:
squareError = tf.reduce_sum(tf.pow(y_conv - y_, 2))
train_step = tf.train.AdamOptimizer(1e-4).minimize(squareError)
accuracy = tf.reduce_mean(tf.abs(y_conv - y_))

sess.run(tf.initialize_all_variables())

batchSize = 20

numBatches = len(imagesCNNTrain)/batchSize

for i in range(1000):

  lowerLimit = (i%numBatches)*batchSize
  upperLimit = (i%numBatches+1)*batchSize

  imageBatch = imagesCNNTrain[lowerLimit:upperLimit]
  labelBatch = labelsCNNTrain[lowerLimit:upperLimit]
  
  if i%10 == 0:
    #print imageBatch[0]
    #print len(imageBatch[0])
    #print(len(labelBatch))
    train_accuracy = accuracy.eval(feed_dict={x:imageBatch, y_: labelBatch, keep_prob: 1.0})
    print("Step %d. Average error: %g pixels"%(i, round(train_accuracy,2)))
  train_step.run(feed_dict={x: imageBatch, y_: labelBatch, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: imagesCNNTest, y_: labelsCNNTest, keep_prob: 1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step 0. Average error: 53.24 pixels
Step 10. Average error: 29.22 pixels
Step 20. Average error: 19.36 pixels
Step 30. Average error: 20.71 pixels
Step 40. Average error: 21.96 pixels
Step 50. Average error: 25.02 pixels
Step 60. Average error: 19.91 pixels
Step 70. Average error: 20.55 pixels
Step 80. Average error: 21.61 pixels
Step 90. Average error: 23.4 pixels
Step 100. Average error: 20.17 pixels
Step 110. Average error: 20.35 pixels
Step 120. Average error: 20.92 pixels
Step 130. Average error: 23.3 pixels
Step 140. Average error: 19.3 pixels
Step 150. Average error: 19.05 pixels
Step 160. Average error: 19.82 pixels
Step 170. Average error: 21.57 pixels
Step 180. Average error: 17.14 pixels
Step 190. Average error: 16.92 pixels
Step 200. Average error: 17.38 pixels
Step 210. Average error: 18.4 pixels
Step 220. Average error: 15.47 pixels
Step 230. Average error: 13.83 pixels
Step 240. Average error: 13.1

## Evaluate the network using the testing data

In [ ]:
#imagesDrawTest
#imagesCNNTest
#labelsDrawTest
#labelsCNNTest

for i in range(len(imagesCNNTest)):
    image = imagesCNNTest[i].copy().reshape((1,pageSize[0]*pageSize[1]))

    #Get the estimate center
    estimate = sess.run([y_conv],feed_dict={x:image, keep_prob: 1.0})
    #print y_conv

    estimate = estimate[0][0]
    print "True center: ", labelsCNNTest[i], ". Estimated center: ", estimate
    
    cv2.rectangle(imagesDrawTest[i], (int(estimate[0]), int(estimate[1])), (int(estimate[2]), int(estimate[3])), (0, 0, 0), 1)

    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Image", imagesDrawTest[i])
    cv2.waitKey(0)
cv2.destroyWindow()

True center:  (21, 39, 39, 57) . Estimated center:  [ 23.01860809  38.70614624  38.64608383  54.43304062]
